In [1]:
from itertools import product

import os
import pandas as pd


base_path = "/scratch/gpfs/jp7427/out/isotropic-decaying-turbulence/128x128/cupy/"

In [6]:
# Loop parameters
M_max_values = [0.01, 0.1, 1, 10, 20, 30, 40, 50]

seeds = range(1, 31)

configs = [
    "MUSCL-Hancock",
    "ZS3",
    "ZS7",
    "ZS3t",
    "ZS7t",
    "MM3/1rev/rtol_1e-5",
    "MM7/1rev/rtol_1e-5",
    "MM3/2revs/rtol_1e-5",
    "MM7/2revs/rtol_1e-5",
    "MM3/3revs/rtol_1e-5",
    "MM7/3revs/rtol_1e-5",
    "MM3/3revs/rtol_1e-3",
    "MM7/3revs/rtol_1e-3",
    "MM3/3revs/rtol_1e-2",
    "MM7/3revs/rtol_1e-2",
]

# read simulation outputs
data = []
for config, M_max in product(configs, M_max_values):
    entry = dict(config=config, M_max=M_max, failed=0)
    for seed in seeds:
        if M_max < 1 and seed > 1:
            continue

        sim_path = f"{base_path}{config}/M_max_{M_max}/seed_{seed:02d}/"

        if not os.path.exists(sim_path):
            raise FileNotFoundError(sim_path)
        elif os.path.exists(sim_path + "error.txt"):
            entry["failed"] += 1
        elif os.path.exists(sim_path + "timings.txt"):
            continue
        else:
            raise FileNotFoundError(f"error.txt or timings.txt in {sim_path}")
    data.append(entry)

df = pd.DataFrame(data)

FileNotFoundError: /scratch/gpfs/jp7427/out/isotropic-decaying-turbulence/128x128/cupy/MM3/2revs/rtol_0/M_max_0.01/seed_01/

In [ ]:
df.pivot(index="config", columns="M_max", values="failed").loc[configs, :]

In [4]:
df_to_table = df[df["M_max"] >= 1]


df_to_table["config"] = pd.Categorical(
    df_to_table["config"], categories=configs, ordered=True
)

print(df_to_table.pivot(index="config", columns="M_max", values="failed").to_latex())

/tmp/ipykernel_3032043/1089603256.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_table["config"] = pd.Categorical(


\begin{tabular}{lrrrrrr}
\toprule
M_max & 1.000000 & 10.000000 & 20.000000 & 30.000000 & 40.000000 & 50.000000 \\
config &  &  &  &  &  &  \\
\midrule
MUSCL-Hancock & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS3 & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS7 & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS3t & 0 & 30 & 30 & 30 & 30 & 30 \\
ZS7t & 0 & 30 & 30 & 30 & 30 & 30 \\
MM3/1rev/rtol_1e-5 & 0 & 27 & 30 & 30 & 30 & 30 \\
MM7/1rev/rtol_1e-5 & 0 & 30 & 30 & 30 & 30 & 30 \\
MM3/2revs/rtol_1e-5 & 0 & 0 & 1 & 2 & 1 & 2 \\
MM7/2revs/rtol_1e-5 & 0 & 0 & 2 & 2 & 0 & 1 \\
MM3/3revs/rtol_1e-5 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM7/3revs/rtol_1e-5 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM3/3revs/rtol_1e-3 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM7/3revs/rtol_1e-3 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM3/3revs/rtol_1e-2 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM7/3revs/rtol_1e-2 & 0 & 0 & 0 & 0 & 0 & 1 \\
\bottomrule
\end{tabular}

